In [1]:
import pandas as pd
import re
import pandas_datareader as dr
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


%store -r

In [2]:
%run PrepareData.ipynb

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0
3,550442977802207232,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
4,550443807834402816,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


(3717964, 7)

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL
3,550802957370159104,AAPL
4,550802855129382912,AAPL


(4336445, 2)

,tweet_id,ticker_symbol
3239577,550442977802207232,TSLA
3239578,550443807834402816,TSLA
3239579,550443808606126081,TSLA
3239580,550443809700851716,TSLA
3239581,550443857142611968,TSLA


(1096868, 2)

,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num
0,550442977802207232,TSLA,ShowDreamCar,1420070807,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1
1,550443807834402816,TSLA,i_Know_First,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
2,550443808606126081,TSLA,aaplstocknews,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
3,550443809700851716,TSLA,iknowfirst,1420071005,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1
4,550443857142611968,TSLA,Cprediction,1420071016,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1


,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num
1096863,1212158772015034369,TSLA,StockJoke,1577836146,"I'm getting wasted on NYE, and thanks to @NHTS...",0,0,0
1096864,1212159099632267268,TSLA,sweetog76,1577836224,$TSLA Very Interesting dude:,0,0,0
1096865,1212159184931717120,TSLA,MelaynaLokosky,1577836245,If only 2020 brought more self-awareness to Te...,0,1,0
1096866,1212159838882533376,TSLA,ShortingIsFun,1577836401,In 2020 I may start Tweeting out positive news...,0,0,1
1096867,1212160015332728833,TSLA,Commuternyc,1577836443,Patiently Waiting for the no twitter sitter tw...,0,0,5


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096868 entries, 0 to 1096867
Data columns (total 8 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   tweet_id       1096868 non-null  int64 
 1   ticker_symbol  1096868 non-null  object
 2   writer         1073202 non-null  object
 3   post_date      1096868 non-null  int64 
 4   body           1096868 non-null  object
 5   comment_num    1096868 non-null  int64 
 6   retweet_num    1096868 non-null  int64 
 7   like_num       1096868 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 75.3+ MB
Stored 'tsla_tweets_df' (DataFrame)
Stored 'tsla_stock_values_df' (DataFrame)


In [3]:
tsla_tweets_df.head()

,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,total_engagement
31,550589781810642945,TSLA,i_Know_First,2015-01-01 20:50:08,Stock Market Outlook: An Average Return Of 26....,0,1,2,3
43,550671812078481408,TSLA,kirillklip,2015-01-02 02:16:05,Let's put some numbers and technical details o...,0,1,2,3
48,550675435030597632,TSLA,markbspiegel,2015-01-02 02:30:29,@markbspiegel Maybe Mrs. Musk decided that $16...,1,1,1,3
50,550680353015627776,TSLA,markbspiegel,2015-01-02 02:50:01,"""A Study In Contrast: Dodge's 707 Horsepower H...",1,1,2,4
57,550703624544718848,TSLA,laurenholmesNYC,2015-01-02 04:22:30,Best Trade: $AAPL; $GOOG $TSLA $YHOO $LNKD I r...,0,1,2,3


In [4]:
# The purpose of this function is to prepare text for lexicon by removing special characters and symbols that are irrelavant for the tweet classification
def cleanTweet(text):
    # Remove '#' hash tag     
    text = re.sub('#', '', text) 
    #Remove @mentions 
    text = re.sub('@[A-Za-z0–9]+', '', text) 
    # Remove RT
    tweet = re.sub('RT[\s]+', '', text) 
    # Remove hyperlink
    tweet = re.sub('https?:\/\/\S+', '', text)  
    return text
def getSentiments(tweet):
    sentiments = analyser.polarity_scores(cleanTweet(tweet))
    # print(sentiments)
    return sentiments["neg"],sentiments["neu"],sentiments["pos"],sentiments["compound"]


In [5]:
analyser = SentimentIntensityAnalyzer()
tsla_tweets_df["polarityScore"] = tsla_tweets_df["body"].apply(lambda tweet : analyser.polarity_scores(cleanTweet(tweet))["compound"] )
# tsla_tweets_df[["neg","neu","pos","compound"]] = tsla_tweets_df["body"].apply(lambda tweet: getSentiments(tweet))
tsla_tweets_df["sentiment"] = tsla_tweets_df["polarityScore"].apply(lambda ps : 1 if(ps >=0.05) else 0 )
display(tsla_tweets_df.head())
display(tsla_tweets_df.tail())
tsla_tweets_df["sentiment"].value_counts()

,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,total_engagement,polarityScore,sentiment
31,550589781810642945,TSLA,i_Know_First,2015-01-01 20:50:08,Stock Market Outlook: An Average Return Of 26....,0,1,2,3,-0.2732,0
43,550671812078481408,TSLA,kirillklip,2015-01-02 02:16:05,Let's put some numbers and technical details o...,0,1,2,3,0.0000,0
48,550675435030597632,TSLA,markbspiegel,2015-01-02 02:30:29,@markbspiegel Maybe Mrs. Musk decided that $16...,1,1,1,3,0.4926,1
50,550680353015627776,TSLA,markbspiegel,2015-01-02 02:50:01,"""A Study In Contrast: Dodge's 707 Horsepower H...",1,1,2,4,0.0000,0
57,550703624544718848,TSLA,laurenholmesNYC,2015-01-02 04:22:30,Best Trade: $AAPL; $GOOG $TSLA $YHOO $LNKD I r...,0,1,2,3,0.6369,1


,tweet_id,ticker_symbol,writer,post_date,body,comment_num,retweet_num,like_num,total_engagement,polarityScore,sentiment
1096856,1212152718640472064,TSLA,decampkev,2020-01-01 10:25:03,Most bulls understand that quite well. In fact...,1,0,3,4,0.5975,1
1096858,1212153830563024898,TSLA,Commuternyc,2020-01-01 10:29:28,Oh that is why Tesla is SO SO profitable! That...,2,0,1,3,0.8741,1
1096859,1212154167185289216,TSLA,Commuternyc,2020-01-01 10:30:48,Nah This will be a trip! Especially without a ...,0,0,3,3,0.7163,1
1096861,1212155937588416517,TSLA,QContrarian,2020-01-01 10:37:50,look at #tesla registrations in Netherlands f...,2,1,0,3,0.0000,0
1096867,1212160015332728833,TSLA,Commuternyc,2020-01-01 10:54:03,Patiently Waiting for the no twitter sitter tw...,0,0,5,5,-0.2960,0


0    206990
1    150552
Name: sentiment, dtype: int64

In [6]:
tsla_sentiments_df = tsla_tweets_df[["post_date","polarityScore"]].copy()
tsla_sentiments_df['Date'] = pd.to_datetime(tsla_tweets_df['post_date'].dt.date, utc=False)
tsla_sentiments_df = tsla_sentiments_df.groupby('Date').mean()
display(tsla_sentiments_df.head())
display(tsla_sentiments_df.tail())
# tsla_sentiments_df.plot(kind="line")

,polarityScore
Date,
2015-01-01,-0.273200
2015-01-02,0.108030
2015-01-03,0.113978
2015-01-04,0.119152
2015-01-05,0.178900


,polarityScore
Date,
2019-12-28,0.113058
2019-12-29,0.103000
2019-12-30,0.055549
2019-12-31,0.042973
2020-01-01,0.178863


In [7]:
%store tsla_sentiments_df

Stored 'tsla_sentiments_df' (DataFrame)
